In [479]:
#Proyecto Shark Attack

In [480]:
import pandas as pd

In [481]:
data = pd.read_csv('GSAF2.csv')


In [482]:
null_cols = data.isnull().sum()
null_cols[null_cols > 0]

Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
href formula                 1
href                         3
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

In [483]:
#Eliminamos las columnas unnamed 
data = data.rename(index=str, columns={"Sex ": "Sex", "Species ": "Species"})
data1 = data.drop(["Unnamed: 22","Unnamed: 23"], axis=1)


In [484]:
column_order = ['Case Number','Case Number.1','Case Number.2','Date','Year','Time','Country','Area','Location','Activity','Type','Name','Sex','Age','Injury','Fatal (Y/N)','Species','Investigator or Source','pdf','href formula','href','original order']
data1 = data1[column_order]




In [485]:
#Eliminamos todos los datos anteriores a 1900
years = data1["Year"]
drop_cols = list(years[years < 1900].index)
data1 = data1.drop(drop_cols, axis=0)


In [486]:
# Eliminamos las columnas Case Number.1 y .2 cuyos valores están duplicados con la columna Case Number
data1 = data1.drop(["Case Number.1","Case Number.2"], axis=1)

In [487]:
#Eliminamos la columna Name ya que no es relevante para el estudio
data1 = data1.drop(["Name"], axis=1)

In [488]:
data1.dtypes

Case Number               object
Date                      object
Year                       int64
Time                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Type                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Species                   object
Investigator or Source    object
pdf                       object
href formula              object
href                      object
original order             int64
dtype: object

In [489]:
data1 = data1.drop(['Investigator or Source','pdf','href formula','href','original order','Injury'], axis=1)

In [490]:
null_cols1 = data1.isnull().sum()
null_cols1[null_cols1 > 0]

Time           2633
Country          22
Area            295
Location        363
Activity        426
Sex             505
Age            2103
Fatal (Y/N)       9
Species        2389
dtype: int64

In [491]:
data1 = data1.dropna(subset=['Country','Fatal (Y/N)','Area','Location'], axis=0)


In [492]:
null_cols1 = data1.isnull().sum()
null_cols1[null_cols1 > 0]

Time        2200
Activity     352
Sex          421
Age         1770
Species     2039
dtype: int64

In [493]:
data1[['Activity','Sex','Species']] = data[['Activity','Sex','Species']].fillna('Unknown')
data1[['Age','Time']] = data[['Age','Time']].fillna('0')

In [494]:
null_cols1 = data1.isnull().sum()
null_cols1[null_cols1 > 0]

Series([], dtype: int64)

In [495]:
data1.loc[data1['Activity'].str.startswith('Surf'), 'Activity'] = 'Surfing'
data1.loc[data1['Activity'].str.startswith('Fish'), 'Activity'] = 'Fishing'
data1.loc[data1['Activity'].str.startswith('Lobs'), 'Activity'] = 'Fishing'
data1.loc[data1['Activity'].str.startswith('Catch'), 'Activity'] = 'Fishing'
data1.loc[data1['Activity'].str.startswith('fish'), 'Activity'] = 'Fishing'
data1.loc[data1['Activity'].str.startswith('Wade'), 'Activity'] = 'Fishing'
data1.loc[data1['Activity'].str.startswith('Harp'), 'Activity'] = 'Fishing'
data1.loc[data1['Activity'].str.startswith('Fly f'), 'Activity'] = 'Fishing'
data1.loc[data1['Activity'].str.startswith('Swim'), 'Activity'] = 'Swiming'
data1.loc[data1['Activity'].str.startswith('Tread'), 'Activity'] = 'Swiming'
data1.loc[data1['Activity'].str.startswith('Float'), 'Activity'] = 'Swiming'
data1.loc[data1['Activity'].str.startswith('Bath'), 'Activity'] = 'Swiming'
data1.loc[data1['Activity'].str.startswith('"Swim"'), 'Activity'] = 'Swiming'
data1.loc[data1['Activity'].str.startswith('Wadi'), 'Activity'] = 'Swiming'
data1.loc[data1['Activity'].str.startswith('Kaya'), 'Activity'] = 'Kayaking'
data1.loc[data1['Activity'].str.startswith('Canoe'), 'Activity'] = 'Kayaking'
data1.loc[data1['Activity'].str.startswith('Scub'), 'Activity'] = 'Diving'
data1.loc[data1['Activity'].str.startswith('SCUB'), 'Activity'] = 'Diving'
data1.loc[data1['Activity'].str.startswith('Kite'), 'Activity'] = 'Kite'
data1.loc[data1['Activity'].str.startswith('Snor'), 'Activity'] = 'Snorkeling'
data1.loc[data1['Activity'].str.startswith('Boat'), 'Activity'] = 'Sailing'
data1.loc[data1['Activity'].str.startswith('Sail'), 'Activity'] = 'Sailing'
data1.loc[data1['Activity'].str.startswith('Free'), 'Activity'] = 'Diving'
data1.loc[data1['Activity'].str.startswith('Div'), 'Activity'] = 'Diving'
data1.loc[data1['Activity'].str.startswith('Skin'), 'Activity'] = 'Diving'
data1.loc[data1['Activity'].str.startswith('Pearl'), 'Activity'] = 'Diving'
data1.loc[data1['Activity'].str.startswith('Spear'), 'Activity'] = 'Fishing'
data1.loc[data1['Activity'].str.startswith('Argentin'), 'Activity'] = 'Air Disaster'
data1.loc[data1['Activity'].str.startswith('Air'), 'Activity'] = 'Air Disaster'
data1.loc[data1['Activity'].str.startswith('"Flying Tiger"'), 'Activity'] = 'Air Disaster'
data1.loc[data1['Activity'].str.startswith('Boeing'), 'Activity'] = 'Air Disaster'
data1.loc[data1['Activity'].str.startswith('Boogie'), 'Activity'] = 'Bodyboarding'
data1.loc[data1['Activity'].str.startswith('Body'), 'Activity'] = 'Bodyboarding'
data1.loc[data1['Activity'].str.startswith('Play'), 'Activity'] = 'Playing'
data1.loc[data1['Activity'].str.startswith('Padd'), 'Activity'] = 'Paddling'
data1.loc[data1['Activity'].str.startswith('Jump'), 'Activity'] = 'Jumping'
data1.loc[data1['Activity'].str.startswith('Shark'), 'Activity'] = 'Shark Fishing'
data1.loc[data1['Activity'].str.startswith('Remov'), 'Activity'] = 'Shark Fishing'
data1.loc[data1['Activity'].str.startswith('Kill'), 'Activity'] = 'Shark Fishing'
data1.loc[data1['Activity'].str.startswith('Stand'), 'Activity'] = 'Standing'
data1.loc[data1['Activity'].str.startswith('Hunt'), 'Activity'] = 'Fishing'
data1.loc[data1['Activity'].str.startswith('No '), 'Activity'] = 'Unknown'
data1.loc[data1['Activity'].str.startswith('.'), 'Activity'] = 'Unknown'
data1.loc[data1['Activity'].str.startswith('Kite'), 'Activity'] = 'Kitesurfing'
data1.loc[data1['Activity'].str.startswith('Sea D'), 'Activity'] = 'Sea Disaster'
data1.loc[data1['Activity'].str.startswith('Ferry'), 'Activity'] = 'Sea Disaster'
data1.loc[data1['Activity'].str.startswith('Wind'), 'Activity'] = 'Windsurfing'
data1.loc[data1['Activity'].str.startswith('Help'), 'Activity'] = 'Helping'
data1.loc[data1['Activity'].str.startswith('Photo'), 'Activity'] = 'Photographing'

In [496]:
datavalue = data1[["Activity"]].stack().value_counts().index.tolist()
display(datavalue)
len(datavalue)

['Swiming',
 'Surfing',
 'Fishing',
 'Diving',
 'Unknown',
 'Bodyboarding',
 'Standing',
 'Snorkeling',
 'Kayaking',
 'Paddling',
 'Sailing',
 'Shark Fishing',
 'Playing',
 'Windsurfing',
 'Jumping',
 'Kitesurfing',
 'Walking',
 'Air Disaster',
 'Rowing',
 'Sitting on surfboard',
 'Sea Disaster',
 'Fell overboard',
 'Splashing',
 'Sea disaster',
 'Lifesaving drill',
 'Seine netting',
 'Walking out of the water after surfing',
 'Photographing',
 'Helping',
 'Hookah diving for abalone',
 'Fell into the water',
 'Escaping from Alacatraz',
 'Their 9 m launch was run down by a 25,000-ton Japanese freighter  on the night of 3-11-1977 & they drifted, clinging to an icebox for 2 days',
 'Hard hat diving',
 'Crabbing',
 'Feeding fish',
 'Dangling feet in the water',
 'Sculling',
 'Crayfishing',
 'Splashing ',
 'Washing',
 'The Christie V sank on 11/6/1988, survivors were adrift on a dinghy',
 'Attempting to kill a shark with explosives',
 'Teasing a shark',
 'Finning the shark that bit him',
 '

453